Suppose you were tasked with ranking a collection of web pages.  

For instance perhaps you were trying to build a search engine, and wanted to order all results that matched a user's query.

We can take advantage of the web's dircted graph structure.

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Directed.svg/250px-Directed.svg.png)

That is, we can consider the "web graph" which has all web pages as nodes and an arrow going from one page to another just in case there's a link from the first to the second.

I couldn't find a picture of the web graph, but here's an artist rendition of the internet graph

http://www.opte.org

Given the graph structure of the internet we might hope to find some quantitative measure of how likely a "random web surfer" (someone just randomly clicking links) is to be visiting a web page after a very long surf session.


![alt text](https://memegenerator.net/img/instances/66405812/so-pitted.jpg)

Let's take a concrete example of a very small web and see if we can formalize this.

In [1]:
web_graph = {
    'a': {'b', 'c'},
  'b': {'b', 'c', 'd'},
  'c': {},
  'd': {'e'},
  'e': {'d'}}

pages = sorted(web_graph.keys())

web_graph

{'a': {'b', 'c'}, 'b': {'b', 'c', 'd'}, 'c': {}, 'd': {'e'}, 'e': {'d'}}

We want to be able to think about arbrarily long sequences of clicking, so we will imagine that if our surfer runs into a deadend like page  𝑐 , they somehow randomly select a webpage and continue.   Thus there's really no difference between a page linking to nothing and linking to every other page.  Let's modify our graph to reflect this.

In [2]:
def remove_dead_ends(graph):
  result = graph.copy()
  for page in graph:
    if len(graph[page])==0:
      result[page]=set(graph.keys())
  return result
  

In [3]:
web_graph_no_dead = remove_dead_ends(web_graph)

web_graph_no_dead

{'a': {'b', 'c'},
 'b': {'b', 'c', 'd'},
 'c': {'a', 'b', 'c', 'd', 'e'},
 'd': {'e'},
 'e': {'d'}}

Now remember that we can represent any graph by an 
adjacency matrix

$$A = \left[ \begin{array}{cccc}
\mathbb{1}_{1, 1} & \mathbb{1}_{1, 2} & \ldots & \mathbb{1}_{1, n} \\
\mathbb{1}_{2, 1} & \mathbb{1}_{2, 2} & \ldots & \mathbb{1}_{2, n} \\
\vdots & \vdots & \ddots & \vdots\\
\mathbb{1}_{n, 1} & \mathbb{1}_{n, 2} & \ldots & \mathbb{1}_{n, n} 
\end{array} \right]$$

where the value  $\mathbb{1}_{i, j}$ (in the $i$th row and $j$th column) is defined to be 1 if there is a link from the $j$th page to the $i$th page and 0 otherwise.  For instance given the graph represented by the adjacency list above

In [4]:
import numpy as np
import pandas as pd

In [5]:
def get_adj_matrix(adj_list): 
  pages = sorted(adj_list.keys())
  A = pd.DataFrame(0, index = pages, columns= pages)
  for source in adj_list:
    for sink in adj_list[source]:
      A.loc[sink, source] = 1
  return A
      
web_matrix = get_adj_matrix(web_graph_no_dead)

web_matrix

,a,b,c,d,e
a,0,0,1,0,0
b,1,1,1,0,0
c,1,1,1,0,0
d,0,1,1,0,1
e,0,0,1,1,0


Given these links, and given that the surfer is randomly clicking links, we can get the condictional probabilities of linking to any page from any page just by normalizing each row by its number of nonzero entries.

In [6]:
def get_transition_matrix(web_matrix):
  result = web_matrix / web_matrix.sum(axis = 0)
  return result

T = get_transition_matrix(web_matrix)

T

,a,b,c,d,e
a,0.0,0.000000,0.2,0.0,0.0
b,0.5,0.333333,0.2,0.0,0.0
c,0.5,0.333333,0.2,0.0,0.0
d,0.0,0.333333,0.2,0.0,1.0
e,0.0,0.000000,0.2,1.0,0.0


Notice a few things:

1.  Each column of $T$ gives a probability distribution (non-negative entries summing to 1). Such matrices are called (column) stochastic.

2. 
Doing a plain old matrix-vector multiplication of $T$ with a "one hot" vector (with a 1 in the place of page $x$) will just give the column corresponding to page $x$, which is the vector of probabilities of being in all possible pages on the next iteration $t+1$ given that one is on page $x$ at time $t$.  

3.  
More generally given any vector $\mathbf{p}$ giving a probability distribution (non-negative entries summing to 1) of being in each of the various pages, the vector 
$\mathbf{p}' = T\mathbf{p}$ is gives the probabilities of being on each of the pages at the next iteration given that one's current location is distributed according to $\mathbf{p}$.  For example:

In [7]:
N= web_matrix.shape[0]
uniform_random_start = pd.Series(np.repeat(1./N, N), index = T.index)
print uniform_random_start

a    0.2
b    0.2
c    0.2
d    0.2
e    0.2
dtype: float64


In [8]:
T.dot(uniform_random_start)

a    0.040000
b    0.206667
c    0.206667
d    0.306667
e    0.240000
dtype: float64

4.  It follows that if the vector $\mathbf{p}$ gives the probability that the surfer is at each of the pages at time $t$, then 
$$\mathbf{p}'' = T(\mathbf{p}') = T(T(\mathbf{p})) = T^{2}\mathbf{p}$$ gives the probability that the surfer is at each of the pages at $t+2$. 



5.  Even more generally, if the vector $\mathbf{p}^{(0)}$ gives the probability that the surfer is at each of the pages at time $t$, then 
$$\mathbf{p}^{(k)} = T^{k}\mathbf{p}$$ gives the probability that the surfer is at each of the pages at $t+k$. 

So perhaps it makes sense to assume that the surfer starts out on any page with equal probability, and let our "scores" be the limit (if it exists) of $\mathbf{p}^{(k)}$ as $k\rightarrow \infty$.

Let's try this out:

In [9]:
def dist(k, M, initial_dist):
  # The distribution of after 
  # k iterations 
  # given a transition matrix T and 
  # starting distribution initial_dist
  dist = np.linalg.matrix_power(M, k).dot(initial_dist)
  print pd.Series(np.round(dist, 4), index = initial_dist.index)

In [10]:
dist(1, T, uniform_random_start)

a    0.0400
b    0.2067
c    0.2067
d    0.3067
e    0.2400
dtype: float64


In [11]:
dist(2, T, uniform_random_start)

a    0.0413
b    0.1302
c    0.1302
d    0.3502
e    0.3480
dtype: float64


In [12]:
dist(3, T, uniform_random_start)

a    0.0260
b    0.0901
c    0.0901
d    0.4175
e    0.3763
dtype: float64


In [13]:
 dist(1000, T, uniform_random_start)

a    0.0000
b    0.0000
c    0.0000
d    0.4884
e    0.5116
dtype: float64


In [14]:
dist(1001, T, uniform_random_start)

a    0.0000
b    0.0000
c    0.0000
d    0.5116
e    0.4884
dtype: float64


We can see that what's happening here is that the surfer is getting trapped in a cycle between pages $d$ and $e$.    Indeed this is a more general case of the "dead ends" that we dealt with at the beginning!

But then we might try to deal with the problem in a similar way:

For instance we might stipulate that the surfer occasionally (say with probability $\alpha$) gets bored clicking links and enters a random page's url into the browser.

We can tweak our transition matrix $T$ above to represent this (while keeping the columns our matrix probability distributions) in the following way:

In [15]:
def dampen(M, alpha):
  N = M.shape[0] # Number of pages
  result = (1-alpha)*M + alpha*(1./N)
  return result

D = dampen(T, alpha = .1)

D

,a,b,c,d,e
a,0.02,0.02,0.2,0.02,0.02
b,0.47,0.32,0.2,0.02,0.02
c,0.47,0.32,0.2,0.02,0.02
d,0.02,0.32,0.2,0.02,0.92
e,0.02,0.02,0.2,0.92,0.02


In [16]:
dist(1000,  D, uniform_random_start)

a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


In [17]:
dist(1001,  D, uniform_random_start)

a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


What if we try a different starting distribution?  Perhaps a one-hot distribution?

In [18]:
start_at_d = pd.Series([0.,0.,0., 1., 0.], index = T.index)

start_at_d

a    0.0
b    0.0
c    0.0
d    1.0
e    0.0
dtype: float64

In [19]:
dist(1000,  D, start_at_d)

a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


In [20]:
dist(1000,  D, uniform_random_start)

a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


Which is the same distribution that we got from the uniform random start.

How interesting.  

Will they always be the same?  

Are there any starting places (or distributions) from which the sequence of vectors does not converge at all?  

Let's bracket this for a moment.

You might have noticed that the vectors of scores $\mathbf{p}^{(k)}=D^{k}\mathbf{p}$ that we were generating above for large $k$ seemed to be "converging," that is, for large $k$, $\mathbf{p}^{(k)}$ was not much different from $\mathbf{p}^{(k+1)} = D\mathbf{p}^{(k)}$.

Said differently, this means that, for large $k$, $\mathbf{p}^{(k)}$ is *almost* a solution to the following equation

$$p^* = Dp^*$$

This is a satisfying result since we might also have the intuition that a page should have a high score if it is linked to by many other pages.  

But not just any pages, high scoring pages!  

Also preferably pages that link to fewer things rather than more things!  

If only we could let each page's score be the weighted sum of all the scores of pages than linked to it, where the weights are determined by how many outgoing links each of those pages has.  You know, something like 

Scores $$s_{i} = \sum_{j\in In(i)}\frac{ s_{j}}{N_{j}} = \sum_{j = 1}^{n} \frac{ \mathbb{1}_{i,j}}{N_{j}}s_{j}$$

with each $s_i>0$.

Of course you can check that, putting all $n$ of these equations together as a single matrix equation gives
 
$ \mathbf{s} = T \mathbf{s}$

where $T$ here is exactly the $T$ from above.

***

Vectors like $\mathbf{s}$ have a special name:  They are called *eigenvectors*.

More formally, given a matrix $M\in \mathbb{R}^{n\times n}$, $\lambda \in \mathbb{R}$ and $\mathbf{v} \in \mathbb{R}^{n}-\{\mathbf{0}\}$ such that $M\mathbf{v} = \lambda\mathbf{v}$, then we say that $\mathbf{v}$ is an eigenvector for $M$ with eigenvalue $\lambda$.  

So it seems that we might want an eigenvector for our transition matrix $T$ for the eigenvalue $\lambda = 1$. 

There are many highly optimized libraries for finding the eigenvectors of a matrix.

In [21]:
def try_compute_unit_eigenvector(M):
  s = [round(x, 4) for x in np.linalg.eig(M)[0]]
  print "The eigenvalues are "
  print s
  print "\n"
  if 1. not in s:
    print "1. is not an eigenvalue"
    return
  else:
    print "A unit eigenvector for the eigenvalue 1. is"
    unit_index = s.index(1.)
    v = np.linalg.eig(M)[1][:,unit_index ]
    v_norm = np.round(v/v.sum(), 4)
    print pd.Series(v_norm, index = M.index)

    
try_compute_unit_eigenvector(T)

The eigenvalues are 
[-0.147, 0.6803, 1.0, -1.0, -0.0]


A unit eigenvector for the eigenvalue 1. is
a   -0.0
b   -0.0
c   -0.0
d    0.5
e    0.5
dtype: float64


So unfortunately in this case we have a solution to $T\mathbf{s} = \mathbf{s}$ but none of them will have all positive components.

So our next question might resonable be

Under what conditions does a matrix have an eigenvector for the eigenvalue $\lambda=1$ such that there is a corresponding eigenvector with all positive entries?

We will not attempt to give necessary and sufficient conditions for this to be the case, but we will state one theorem which says that such a vector does exist for a large family of matrices:

Perron's Theorem:

[See the Wikipedia article](https://en.wikipedia.org/wiki/Perron%E2%80%93Frobenius_theorem)

Let $M\in \mathbb{R}^{n\times n}$ be an $n\times n$ matrix where each component is positive ($M_{i,j}>0$ for $1\leq i \leq n$, $1 \leq j \leq n$). Suppose in addition that $M$ is *column stochastic*, meaning each of its columns sum to 1.  Then

1.  $\lambda = 1$ is an eigenvalue for $M$ and there is exactly one unit-length eigenvector $\mathbf{v}$ (called the *Perron* vector) for $\lambda =1$ with each component positive ($v_{i}>0$ for $1\leq i \leq n$).

2.  The powers $M^k$ converge to the the following projection $P:\mathbb{R}^{ n} \rightarrow span(\mathbf{v})$:

$$lim_{k\rightarrow \infty}M^{k} = P = [\mathbf{v} ~ \mathbf{v} \ldots \mathbf{v}] = \left[ \begin{array}{cccc}
v_{1} & v_{1} & \ldots & v_{1} \\
v_{2} & v_{2} & \ldots & v_{2} \\
\vdots & \vdots & \ddots & \vdots\\
v_{n} & v_{n} & \ldots & v_{n} 
\end{array} \right]$$


In particular if follows that if $\mathbf{p}^{(0)}\in \mathbb{R}^{n}$ is *any* distribution vector (non-negative components summing to 1.), then

$$\lim_{k\rightarrow \infty}\mathbf{p}^{(k)}=  \lim_{k\rightarrow \infty}M^{k} \mathbf{p}^{(0)}= P \mathbf{p}^{(0)} =  \mathbf{v}.$$

This gives answers to the reliability of the random surfer method as well as telling us that our damped matrices will admit an appropriate eigenvector to be used as scores, namely

Given any damped (positive) matrix $D$ for the web graph: 

1.  The limiting distribution $\lim_{k\rightarrow \infty}\mathbf{p}^{(k)}$ of the random surfer exists,

2.  It will be the same regardless of the random surfer's starting distribution $\mathbf{p}^{(0)}$,

3. It will be equal to the unique unit length eigenvector for $D$ for the eigenvalue $\lambda =1$ that has positive entries.

In [22]:
try_compute_unit_eigenvector(D)

The eigenvalues are 
[-0.9, 1.0, 0.6123, -0.1323, 0.0]


A unit eigenvector for the eigenvalue 1. is
a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


In [23]:
dist(1000, D, uniform_random_start)

a    0.0319
b    0.0661
c    0.0661
d    0.4232
e    0.4128
dtype: float64


Discussion:  How would *you* compute $\lim_{k\rightarrow \infty}\mathbf{p}^{(k)}$ ?

![alt text](https://i.imgur.com/HBvOvkP.jpg)